In [1]:
import pandas as pd
import numpy as np

In [2]:
df_vcr_c = pd.read_csv('dataset_vcr_compact.csv')
df_vcr_c = df_vcr_c[df_vcr_c['monto'] < 56000].copy()
df_vcr_c['log_monto']=np.log(df_vcr_c['monto'])
df_vcr_c['log_monto'].describe()

df_vcr_e = pd.read_csv('dataset_vcr_expanded.csv')
df_vcr_e = df_vcr_e[df_vcr_e['monto'] < 56000].copy()
df_vcr_e['log_monto']=np.log(df_vcr_e['monto'])
df_vcr_e['log_monto'].describe()

count    25211.000000
mean         8.395828
std          0.830310
min          5.950643
25%          7.740664
50%          8.242756
75%          8.984694
max         10.915088
Name: log_monto, dtype: float64

In [3]:
df_vcr_c.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 25211 entries, 0 to 25214
Data columns (total 41 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          25211 non-null  int64  
 1   monto                       25211 non-null  int64  
 2   superficie_t                25211 non-null  float64
 3   dormitorios                 25211 non-null  int64  
 4   dormitorios_faltante        25211 non-null  int64  
 5   banos                       25211 non-null  int64  
 6   banos_faltante              25211 non-null  int64  
 7   antiguedad                  25211 non-null  int64  
 8   antiguedad_faltante         25211 non-null  int64  
 9   Or_N                        25211 non-null  int64  
 10  Or_S                        25211 non-null  int64  
 11  Or_E                        25211 non-null  int64  
 12  Or_O                        25211 non-null  int64  
 13  Or_Faltante                 25211 no

In [4]:
df_vcr_e.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 25211 entries, 0 to 25214
Data columns (total 197 columns):
 #    Column                        Non-Null Count  Dtype  
---   ------                        --------------  -----  
 0    id                            25211 non-null  int64  
 1    monto                         25211 non-null  int64  
 2    superficie_t                  25211 non-null  float64
 3    dormitorios                   25211 non-null  int64  
 4    dormitorios_faltante          25211 non-null  int64  
 5    banos                         25211 non-null  int64  
 6    banos_faltante                25211 non-null  int64  
 7    antiguedad                    25211 non-null  int64  
 8    antiguedad_faltante           25211 non-null  int64  
 9    Or_N                          25211 non-null  int64  
 10   Or_S                          25211 non-null  int64  
 11   Or_E                          25211 non-null  int64  
 12   Or_O                          25211 non-null  int

### Primer modelo --> Control: Modelo Hedonico ElasticNet. 
 - Dataset sin elementos georreferenciales (latitud, longitud, vcr)

In [5]:
df_control_1 =df_vcr_c.copy()
obj_cols = df_control_1.select_dtypes(include=["object"]).columns
cols_to_drop = list(obj_cols) + ["id", "latitud", "longitud"]
df_control_1 = df_control_1.drop(columns=cols_to_drop)
df_control_1.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 25211 entries, 0 to 25214
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   monto                 25211 non-null  int64  
 1   superficie_t          25211 non-null  float64
 2   dormitorios           25211 non-null  int64  
 3   dormitorios_faltante  25211 non-null  int64  
 4   banos                 25211 non-null  int64  
 5   banos_faltante        25211 non-null  int64  
 6   antiguedad            25211 non-null  int64  
 7   antiguedad_faltante   25211 non-null  int64  
 8   Or_N                  25211 non-null  int64  
 9   Or_S                  25211 non-null  int64  
 10  Or_E                  25211 non-null  int64  
 11  Or_O                  25211 non-null  int64  
 12  Or_Faltante           25211 non-null  int64  
 13  terraza               25211 non-null  float64
 14  estacionamiento       25211 non-null  int64  
 15  bodegas               25

In [ ]:
# Imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configuración
RANDOM_STATE = 42
TEST_SIZE = 0.2

# Columna objetivo
target_col = "log_monto"

# Evitar fuga: nunca usar 'monto' ni 'log_monto' como features
X = df_control_1.drop(columns=["monto", "log_monto"])
y = df_control_1[target_col].values

# Guardamos índices para reconstruir métricas en espacio de precio cuando entrenamos en log
indices = np.arange(len(df_control_1))
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, indices, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

# Definición del modelo
# Escalado es crucial para ElasticNet; CV selecciona alpha y l1_ratio.
model = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        (
            "enet",
            ElasticNetCV(#Busqueda automatica de hiperparametros por validacion cruzada
                l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 1.0],# 1=Lasso puro, 0=Ridge puro
                cv=5, #5-Fold CV
                random_state=RANDOM_STATE,
                n_alphas=100,
                max_iter=10000,
                #Para cada l1_ratio se genera un camino de regularizacion con 100 valores de alpha
                #Mediante la CV se elige en mejor alpha
            ),
        ),
    ]
)
# Tras encontrar todos los hiperparametros optimos, el estimador se reentrena con todo el conjunto 
# de entrenamiento con esos hiperparametros. 

# Entrenamiento y evaluación. Se ejecuta el pipeline anterior.
model.fit(X_train, y_train)
y_pred = model.predict(X_test) #Incluye el escalado aprendido a partir del set de entrenamiento

# Volvemos a precio para métricas interpretables
y_true_price = df_control_1.loc[idx_test, "monto"].values.astype(float)
y_pred_price = np.exp(y_pred)
r2_log = r2_score(y_test, y_pred)

rmse = mean_squared_error(y_true_price, y_pred_price, squared=False)
mae = mean_absolute_error(y_true_price, y_pred_price)
mape = np.mean(np.abs((y_true_price - y_pred_price) / np.clip(y_true_price, 1e-9, None))) * 100

print("=== ElasticNet Hedónico (baseline) ===")
print(f"R^2 (log espacio): {r2_log:.4f}")
print(f"RMSE (precio): {rmse:,.2f}")
print(f"MAE  (precio): {mae:,.2f}")
print(f"MAPE (%): {mape:.2f}")

# Extra: coeficientes
enet: ElasticNetCV = model.named_steps["enet"]
coef = enet.coef_
feat_names = X.columns
coef_df = (
    pd.DataFrame({"feature": feat_names, "coef": coef})
    .assign(abs_coef=lambda d: d.coef.abs())
    .sort_values("abs_coef", ascending=False)
)
print("\nTop 20 coeficientes por |coef| (estandarizados):")
print(coef_df.head(20).to_string(index=False))

# Exponer objetos útiles
best_alpha = enet.alpha_
best_l1_ratio = enet.l1_ratio_
print(f"\nMejores hiperparámetros -> alpha: {best_alpha:.6f}, l1_ratio: {best_l1_ratio}")

=== ElasticNet Hedónico (baseline) ===
R^2 (log espacio): 0.8092
RMSE (precio): 7,443.81
MAE  (precio): 3,838.62
MAPE (%): 70.37

Top 20 coeficientes por |coef| (estandarizados):
             feature      coef  abs_coef
        superficie_t  0.456210  0.456210
               banos  0.168899  0.168899
             bodegas  0.124082  0.124082
     estacionamiento  0.078359  0.078359
      banos_faltante  0.070199  0.070199
         dormitorios  0.030630  0.030630
   flag_Monoambiente -0.021048  0.021048
                Or_E  0.018385  0.018385
          antiguedad -0.016550  0.016550
   flag_Departamento  0.014535  0.014535
 antiguedad_faltante -0.012373  0.012373
        flag_Premium -0.010521  0.010521
                Or_O -0.007520  0.007520
                Or_S -0.006176  0.006176
dormitorios_faltante  0.005869  0.005869
           flag_Loft  0.004791  0.004791
     flag_Multinivel -0.000889  0.000889
         Or_Faltante -0.000662  0.000662
             terraza  0.000259  0.000259
 

c:\Users\Pc-ADS\.conda\envs\tf\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### Segundo modelo --> Control: Modelo Hedonico ElasticNet. 
 - Dataset sin VCR, con latitud y longitud

In [7]:
df_control_2 =df_vcr_c.copy() 
obj_cols = df_control_2.select_dtypes(include=["object"]).columns 
cols_to_drop = list(obj_cols) 
cols_to_drop.append("id") 
df_control_2 = df_control_2.drop(columns=cols_to_drop) 
df_control_2.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 25211 entries, 0 to 25214
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   monto                 25211 non-null  int64  
 1   superficie_t          25211 non-null  float64
 2   dormitorios           25211 non-null  int64  
 3   dormitorios_faltante  25211 non-null  int64  
 4   banos                 25211 non-null  int64  
 5   banos_faltante        25211 non-null  int64  
 6   antiguedad            25211 non-null  int64  
 7   antiguedad_faltante   25211 non-null  int64  
 8   Or_N                  25211 non-null  int64  
 9   Or_S                  25211 non-null  int64  
 10  Or_E                  25211 non-null  int64  
 11  Or_O                  25211 non-null  int64  
 12  Or_Faltante           25211 non-null  int64  
 13  terraza               25211 non-null  float64
 14  estacionamiento       25211 non-null  int64  
 15  bodegas               25

In [8]:
# Imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configuración
RANDOM_STATE = 42
TEST_SIZE = 0.2

# Columna objetivo
target_col = "log_monto"

# Evitar fuga: nunca usar 'monto' ni 'log_monto' como features
X = df_control_2.drop(columns=["monto", "log_monto"])
y = df_control_2[target_col].values

# Guardamos índices para reconstruir métricas en espacio de precio cuando entrenamos en log
indices = np.arange(len(df_control_2))
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, indices, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

# %% Definición del modelo
# Escalado es crucial para ElasticNet; CV selecciona alpha y l1_ratio.
model = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        (
            "enet",
            ElasticNetCV(
                l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 1.0],
                cv=5,
                random_state=RANDOM_STATE,
                n_alphas=100,
                max_iter=10000,
            ),
        ),
    ]
)

# %% Entrenamiento y evaluación
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Volvemos a precio para métricas interpretables
y_true_price = df_control_2.loc[idx_test, "monto"].values.astype(float)
y_pred_price = np.exp(y_pred)
r2_log = r2_score(y_test, y_pred)

rmse = mean_squared_error(y_true_price, y_pred_price, squared=False)
mae = mean_absolute_error(y_true_price, y_pred_price)
mape = np.mean(np.abs((y_true_price - y_pred_price) / np.clip(y_true_price, 1e-9, None))) * 100

print("=== ElasticNet Hedónico (baseline) + lat/lon ===")
print(f"R^2 (log espacio): {r2_log:.4f}")
print(f"RMSE (precio): {rmse:,.2f}")
print(f"MAE  (precio): {mae:,.2f}")
print(f"MAPE (%): {mape:.2f}")

# Extra: coeficientes
enet: ElasticNetCV = model.named_steps["enet"]
coef = enet.coef_
feat_names = X.columns
coef_df = (
    pd.DataFrame({"feature": feat_names, "coef": coef})
    .assign(abs_coef=lambda d: d.coef.abs())
    .sort_values("abs_coef", ascending=False)
)
print("\nTop 20 coeficientes por |coef| (estandarizados):")
print(coef_df.head(20).to_string(index=False))

# Exponer objetos útiles
best_alpha = enet.alpha_
best_l1_ratio = enet.l1_ratio_
print(f"\nMejores hiperparámetros -> alpha: {best_alpha:.6f}, l1_ratio: {best_l1_ratio}")

=== ElasticNet Hedónico (baseline) + lat/lon ===
R^2 (log espacio): 0.8719
RMSE (precio): 6,573.56
MAE  (precio): 3,555.96
MAPE (%): 67.18

Top 20 coeficientes por |coef| (estandarizados):
             feature      coef  abs_coef
        superficie_t  0.325243  0.325243
            longitud  0.238813  0.238813
               banos  0.111769  0.111769
             bodegas  0.083584  0.083584
             latitud  0.083507  0.083507
         dormitorios  0.069636  0.069636
      banos_faltante  0.045834  0.045834
     estacionamiento  0.044259  0.044259
   flag_Monoambiente -0.020912  0.020912
          antiguedad -0.018632  0.018632
                Or_E  0.014820  0.014820
   flag_Departamento  0.010673  0.010673
 antiguedad_faltante -0.007176  0.007176
         Or_Faltante  0.004459  0.004459
           flag_Loft  0.004135  0.004135
        flag_Premium -0.004127  0.004127
dormitorios_faltante  0.003704  0.003704
                Or_N  0.002362  0.002362
     flag_Multinivel -0.002213  

c:\Users\Pc-ADS\.conda\envs\tf\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### Tercer modelo --> Control: Modelo Hedonico ElasticNet. 
 - Dataset completo
 - Se imputan los datos faltantes en las columnas de VCR expandidas. Ver lógica en documentación.

In [9]:
df_control_vcr =df_vcr_e.copy()
obj_cols = df_control_vcr.select_dtypes(include=["object"]).columns
cols_to_drop = list(obj_cols)
cols_to_drop.append("id")
df_control_vcr = df_control_vcr.drop(columns=cols_to_drop)
df_control_vcr.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 25211 entries, 0 to 25214
Data columns (total 181 columns):
 #    Column                        Dtype  
---   ------                        -----  
 0    monto                         int64  
 1    superficie_t                  float64
 2    dormitorios                   int64  
 3    dormitorios_faltante          int64  
 4    banos                         int64  
 5    banos_faltante                int64  
 6    antiguedad                    int64  
 7    antiguedad_faltante           int64  
 8    Or_N                          int64  
 9    Or_S                          int64  
 10   Or_E                          int64  
 11   Or_O                          int64  
 12   Or_Faltante                   int64  
 13   terraza                       float64
 14   estacionamiento               int64  
 15   bodegas                       int64  
 16   flag_Departamento             int64  
 17   flag_Multinivel               int64  
 18   flag_Semi

In [10]:
import re
import numpy as np
import pandas as pd
from typing import Dict, Tuple, Optional

# %% Configuración (
# Dimensiones (1..12) 
DIMS_MAP = {
    1: "count_pois",
    2: "mean_distance",
    3: "min_distance",
    4: "max_distance",
    5: "median_distance",
    6: "std_distance",
    7: "mean_inverse_distance",
    8: "max_inverse_distance",
    9: "sum_inverse_distance",
    10: "ratio_within_near_radius",
    11: "ratio_within_mid_radius",
    12: "ratio_within_far_radius",
}

# Rol por dimensión (para decidir la imputación semántica)
DIM_ROLE = {
    1: "count",                # -> 0
    2: "distance",             # -> R3
    3: "distance",             # -> R3
    4: "distance",             # -> R3
    5: "distance",             # -> R3
    6: "std",                  # -> 0
    7: "inverse",              # -> 0
    8: "inverse",              # -> 0
    9: "inverse",              # -> 0
    10: "ratio",               # -> 0
    11: "ratio",               # -> 0
    12: "ratio",               # -> 0
}

# R3 por tipo de clase
R3_DEFAULT = 2400.0  # clases generales
R3_METRO = 1600.0
R3_BUS = 800.0

# %% Funciones
def _class_and_dim(col: str) -> Optional[Tuple[str, int]]:
    """Extrae (clase, índice de dimensión) de columnas tipo '<clase>_dimXX'."""
    m = re.match(r"^(?P<klass>.+)_dim(?P<idx>\d{1,2})$", col)
    if not m:
        return None
    return m.group("klass"), int(m.group("idx"))


def _r3_for_class(klass: str) -> float:
    k = klass.lower()
    if "metro" in k:
        return R3_METRO
    if "bus" in k:
        return R3_BUS
    return R3_DEFAULT


def impute_vcr_semantic(df: pd.DataFrame) -> pd.DataFrame:
    """Imputa VCR por semántica de ausencia: distancias=R3, inversas/ratios=0, count=0, std=0.
    Además agrega flags `has_<clase>` indicando presencia de POIs por clase.
    """
    out = df.copy()

    # Agrupar columnas por clase
    groups: Dict[str, Dict[int, str]] = {}
    vcr_cols = []
    for c in out.columns:
        parsed = _class_and_dim(c)
        if parsed is None:
            continue
        klass, idx = parsed
        groups.setdefault(klass, {})[idx] = c
        vcr_cols.append(c)

    if not groups:
        # Nada que imputar
        return out

    # Flags de presencia por clase (antes de imputar)
    for klass, dim_map in groups.items():
        cols = list(dim_map.values())
        has_series = out[cols].notna().any(axis=1).astype("int64")
        out[f"has_{klass}"] = has_series  # por qué: distingue ausencia real vs lejanía

    # Imputación por clase/dim
    n_total_nans = int(out[vcr_cols].isna().sum().sum())
    for klass, dim_map in groups.items():
        r3 = _r3_for_class(klass)
        for idx, col in dim_map.items():
            role = DIM_ROLE.get(idx)
            if role == "distance":
                fill_value = r3
            elif role in {"inverse", "ratio", "std", "count"}:
                fill_value = 0.0
            else:
                # Si hay una dimensión desconocida, ser conservador con 0.0
                fill_value = 0.0
            out[col] = out[col].fillna(fill_value)

    n_after_nans = int(out[vcr_cols].isna().sum().sum())
    print(f"Imputación VCR completada. NaNs antes: {n_total_nans:,d} -> después: {n_after_nans:,d}")

    return out



df_control_vcr_imp = impute_vcr_semantic(df_control_vcr)
df_control_vcr_imp.info()  


Imputación VCR completada. NaNs antes: 246,228 -> después: 0
<class 'pandas.core.frame.DataFrame'>
Index: 25211 entries, 0 to 25214
Columns: 194 entries, monto to has_bus
dtypes: float64(161), int64(33)
memory usage: 37.5 MB


In [11]:
# %% Imports
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# %% Configuración
RANDOM_STATE = 42
TEST_SIZE = 0.2
# Paraleliza la CV en todos los cores.
N_JOBS = 4
L1_GRID = [0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 1.0]
N_ALPHAS = 100
CV_FOLDS = 5


# No usar 'monto' ni 'log_monto' como features
X = df_control_vcr_imp.drop(columns=["monto", "log_monto"])
y = df_control_vcr_imp["log_monto"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

print(f"Shape X_train: {X_train.shape}, X_test: {X_test.shape}")

# %% Definición del modelo
model = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        (
            "enet",
            ElasticNetCV(
                l1_ratio=L1_GRID,
                cv=CV_FOLDS,
                random_state=RANDOM_STATE,
                n_alphas=N_ALPHAS,
                max_iter=10000,
                n_jobs=N_JOBS,
            ),
        ),
    ]
)

# %% Entrenamiento y evaluación
start = time.perf_counter()
model.fit(X_train, y_train)
elapsed = time.perf_counter() - start

# Predicción en log y en precio
y_pred_log = model.predict(X_test)
y_true_price = df_control_vcr_imp.loc[y_test.index if hasattr(y_test, 'index') else X_test.index, "monto"] if isinstance(y_test, pd.Series) else df_control_vcr.loc[X_test.index, "monto"].values.astype(float)
y_pred_price = np.exp(y_pred_log)

# Métricas
r2_log = r2_score(y_test, y_pred_log)
rmse = mean_squared_error(y_true_price, y_pred_price, squared=False)
mae = mean_absolute_error(y_true_price, y_pred_price)
mape = np.mean(np.abs((y_true_price - y_pred_price) / np.clip(y_true_price, 1e-9, None))) * 100

print("=== ElasticNet con VCR (control) ===")
print(f"Entrenamiento + CV: {elapsed:.2f} s")
print(f"R^2 (log): {r2_log:.4f}")
print(f"RMSE ($): {rmse:,.2f}")
print(f"MAE  ($): {mae:,.2f}")
print(f"MAPE (%): {mape:.2f}")

# Coeficientes
enet: ElasticNetCV = model.named_steps["enet"]
coef = enet.coef_
feat_names = X.columns
coef_df = (
    pd.DataFrame({"feature": feat_names, "coef": coef})
    .assign(abs_coef=lambda d: d.coef.abs())
    .sort_values("abs_coef", ascending=False)
)
print("\nTop 20 coeficientes por |coef| (estandarizados):")
print(coef_df.head(20).to_string(index=False))

best_alpha = enet.alpha_
best_l1_ratio = enet.l1_ratio_
print(f"\nMejores hiperparámetros -> alpha: {best_alpha:.6f}, l1_ratio: {best_l1_ratio}")


Shape X_train: (20168, 192), X_test: (5043, 192)
=== ElasticNet con VCR (control) ===
Entrenamiento + CV: 46.43 s
R^2 (log): 0.9145
RMSE ($): 2,632.45
MAE  ($): 1,273.44
MAPE (%): 19.24

Top 20 coeficientes por |coef| (estandarizados):
                     feature      coef  abs_coef
                superficie_t  0.318392  0.318392
                    longitud  0.162805  0.162805
 food_and_drink_stores_dim00 -0.106951  0.106951
                     latitud  0.101343  0.101343
               medical_dim00  0.091573  0.091573
                       banos  0.086487  0.086487
     sport_and_leisure_dim00  0.075404  0.075404
                 dormitorios  0.072139  0.072139
        food_and_drink_dim00  0.061614  0.061614
arts_and_entertainment_dim08 -0.056728  0.056728
arts_and_entertainment_dim00 -0.053986  0.053986
                     bodegas  0.048651  0.048651
            veterinary_dim00  0.046458  0.046458
                  antiguedad -0.045243  0.045243
              religion_dim09 

c:\Users\Pc-ADS\.conda\envs\tf\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### Comparación de resultados entre modelos

Los resultados muestran un progreso consistente al incorporar distintos niveles de información georreferencial en el modelo hedónico. El modelo base, construido únicamente con atributos estructurales de las viviendas, alcanzó un R² de 0.81, con un RMSE de 7,444, un MAE de 3,839 y un MAPE de 70.4%, evidenciando una capacidad predictiva limitada. Al incluir latitud y longitud, el desempeño mejora de forma significativa, elevando el R² a 0.87 y reduciendo los errores (RMSE = 6,574; MAE = 3,556; MAPE = 67.2%), lo que confirma la importancia de la ubicación geográfica aun en su forma más simple. Finalmente, la incorporación completa de los vectores de características de referencia (VCR), que capturan métricas sobre el contexto urbano y la presencia de distintos puntos de interés, produce un salto sustancial en la calidad del modelo: R² = 0.91, RMSE = 2,632, MAE = 1,273 y MAPE = 19.2%. Esta mejora no solo refleja una reducción drástica en los errores absolutos y relativos, sino que también demuestra de manera cuantitativa el valor del contexto geoespacial en la predicción de precios inmobiliarios.


| Modelo                                 | R² (log) | RMSE ($) | MAE ($) | MAPE (%) |
|----------------------------------------|:--------:|---------:|--------:|---------:|
| 1) Base (solo estructural)             |  0.8092  |  7,443.81| 3,838.62|    70.37 |
| 2) Base + latitud/longitud             |  0.8719  |  6,573.56| 3,555.96|    67.18 |
| 3) Con VCR completos (contexto urbano) |  0.9145  |  2,632.45| 1,273.44|    19.24 |
